## Note on Simulation Results

The plots presented in this report illustrate the results of one specific run of the simulation model. Due to the stochastic nature of the model, which incorporates random elements such as agent initialization, movement, and resource availability, the outcomes may vary with each execution. 

Therefore, while these plots provide insights into the dynamics of the wolf-sheep population model, they should be interpreted as examples of potential scenarios rather than definitive results. Repeating the simulation multiple times can yield different trajectories and behaviors, highlighting the variability and complexity of the model.


### Persistence of Running Logs and Plots

It is important to note that running logs and plots generated during the simulation in Jupyter notebooks may not persist after the notebook is closed and reopened. This behavior is due to the following reason.

**State Management**: Jupyter does not automatically retain the state of widgets or variable outputs upon reopening a notebook. Each time the notebook is opened, it starts a new session, and any previously generated outputs need to be re-executed to regenerate plots and logs.

## Iteration 1

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# Parameters for the simulation (with default values)
GRID_SIZE = 50
INITIAL_SHEEP = 100
INITIAL_WOLVES = 30
GRASS_REGROWTH_RATE = 0.05
SHEEP_ENERGY_GAIN = 4
WOLF_ENERGY_GAIN = 10
SHEEP_REPRODUCTION_RATE = 0.04
WOLF_REPRODUCTION_RATE = 0.05
ENERGY_LOSS_RATE = 1
SIMULATION_STEPS = 200

# Agent Classes
class Sheep:
    def __init__(self, energy_gain):
        self.x = random.randint(0, GRID_SIZE - 1)
        self.y = random.randint(0, GRID_SIZE - 1)
        self.energy = random.randint(5, 10)
        self.energy_gain = energy_gain

    def move(self):
        self.x = (self.x + random.choice([-1, 0, 1])) % GRID_SIZE
        self.y = (self.y + random.choice([-1, 0, 1])) % GRID_SIZE

    def eat_grass(self, grass):
        if grass[self.x][self.y]:
            grass[self.x][self.y] = False
            self.energy += self.energy_gain

class Wolf:
    def __init__(self, energy_gain):
        self.x = random.randint(0, GRID_SIZE - 1)
        self.y = random.randint(0, GRID_SIZE - 1)
        self.energy = random.randint(10, 20)
        self.energy_gain = energy_gain

    def move(self):
        self.x = (self.x + random.choice([-1, 0, 1])) % GRID_SIZE
        self.y = (self.y + random.choice([-1, 0, 1])) % GRID_SIZE

    def eat_sheep(self, sheep_list):
        for sheep in sheep_list:
            if self.x == sheep.x and self.y == sheep.y:
                sheep_list.remove(sheep)
                self.energy += self.energy_gain
                return True
        return False

# Initialize environment and agents
def initialize(initial_sheep, initial_wolves, sheep_energy_gain, wolf_energy_gain):
    grass = np.random.choice([True, False], (GRID_SIZE, GRID_SIZE), p=[0.5, 0.5])
    sheep_list = [Sheep(sheep_energy_gain) for _ in range(initial_sheep)]
    wolf_list = [Wolf(wolf_energy_gain) for _ in range(initial_wolves)]
    return grass, sheep_list, wolf_list

# Update function
def update(grass, sheep_list, wolf_list, grass_regrowth_rate, sheep_reproduction_rate, wolf_reproduction_rate, energy_loss_rate):
    # Grass regrowth
    grass_regrowth = np.random.choice([True, False], (GRID_SIZE, GRID_SIZE), p=[grass_regrowth_rate, 1 - grass_regrowth_rate])
    grass = np.logical_or(grass, grass_regrowth)

    # Sheep move, eat grass, lose energy, and reproduce
    for sheep in sheep_list[:]:
        sheep.move()
        sheep.eat_grass(grass)
        sheep.energy -= energy_loss_rate
        if sheep.energy <= 0:
            sheep_list.remove(sheep)
        elif random.random() < sheep_reproduction_rate:
            sheep_list.append(Sheep(sheep.energy_gain))

    # Wolves move, eat sheep, lose energy, and reproduce
    for wolf in wolf_list[:]:
        wolf.move()
        if not wolf.eat_sheep(sheep_list):
            wolf.energy -= energy_loss_rate
        if wolf.energy <= 0:
            wolf_list.remove(wolf)
        elif random.random() < wolf_reproduction_rate:
            wolf_list.append(Wolf(wolf.energy_gain))

    return grass, sheep_list, wolf_list

# Simulation
def simulate(steps, grid_size, initial_sheep, initial_wolves, grass_regrowth_rate, sheep_energy_gain, wolf_energy_gain, 
             sheep_reproduction_rate, wolf_reproduction_rate, energy_loss_rate):
    
    global GRID_SIZE
    GRID_SIZE = grid_size

    grass, sheep_list, wolf_list = initialize(initial_sheep, initial_wolves, sheep_energy_gain, wolf_energy_gain)

    sheep_population = []
    wolf_population = []
    grass_coverage = []
    
    for step in range(steps):
        grass, sheep_list, wolf_list = update(grass, sheep_list, wolf_list, grass_regrowth_rate, sheep_reproduction_rate, 
                                              wolf_reproduction_rate, energy_loss_rate)
        sheep_population.append(len(sheep_list))
        wolf_population.append(len(wolf_list))
        grass_coverage.append(np.sum(grass))

        # Log every 10 steps instead of every step
        if step % 10 == 0:
            print(f'Step {step}: Sheep = {len(sheep_list)}, Wolves = {len(wolf_list)}, Grass Coverage = {np.sum(grass)}')

    return sheep_population, wolf_population, grass_coverage

# Plot Population
def plot_population_dynamics(sheep_population, wolf_population, grass_coverage, filename=None):
    plt.figure(figsize=(10, 6))
    plt.plot(sheep_population, label="Sheep Population", color='green')
    plt.plot(wolf_population, label="Wolf Population", color='red')
    plt.plot(grass_coverage, label="Grass Coverage", color='blue')
    plt.xlabel("Time Steps")
    plt.ylabel("Population/Grass Coverage")
    plt.title("Wolf-Sheep Population Dynamics with Grass")
    plt.legend()
    plt.grid()
    
    # Save the figure
    if filename:
        plt.savefig(filename, dpi=300)
        print(f"Plot saved as {filename}")

    plt.show()

# Update the run_simulation func
def run_simulation(grid_size, initial_sheep, initial_wolves, grass_regrowth_rate, sheep_energy_gain, wolf_energy_gain, 
                   sheep_reproduction_rate, wolf_reproduction_rate, energy_loss_rate, filename=None):
    sheep_population, wolf_population, grass_coverage = simulate(SIMULATION_STEPS, grid_size, initial_sheep, initial_wolves, 
                                                                 grass_regrowth_rate, sheep_energy_gain, wolf_energy_gain, 
                                                                 sheep_reproduction_rate, wolf_reproduction_rate, energy_loss_rate)
    plot_population_dynamics(sheep_population, wolf_population, grass_coverage, filename)

# Define sliders
grid_size_slider = widgets.IntSlider(value=50, min=10, max=100, step=1, description='Grid Size')
initial_sheep_slider = widgets.IntSlider(value=100, min=10, max=200, step=1, description='Initial Sheep')
initial_wolves_slider = widgets.IntSlider(value=30, min=5, max=100, step=1, description='Initial Wolves')
grass_regrowth_slider = widgets.FloatSlider(value=0.05, min=0.01, max=0.1, step=0.01, description='Grass Regrowth')
sheep_energy_gain_slider = widgets.IntSlider(value=4, min=1, max=10, step=1, description='Sheep Energy Gain')
wolf_energy_gain_slider = widgets.IntSlider(value=10, min=5, max=20, step=1, description='Wolf Energy Gain')
sheep_reproduction_slider = widgets.FloatSlider(value=0.04, min=0.01, max=0.1, step=0.01, description='Sheep Reproduction')
wolf_reproduction_slider = widgets.FloatSlider(value=0.05, min=0.01, max=0.1, step=0.01, description='Wolf Reproduction')
energy_loss_slider = widgets.FloatSlider(value=1, min=0.5, max=10, step=0.05, description='Energy Loss')  # Updated here

# Create a filename text input
filename_textbox = widgets.Text(value='population_dynamics_iter1.png', description='Filename:')

ui = widgets.VBox([
    grid_size_slider, initial_sheep_slider, initial_wolves_slider, grass_regrowth_slider, 
    sheep_energy_gain_slider, wolf_energy_gain_slider, sheep_reproduction_slider, 
    wolf_reproduction_slider, energy_loss_slider
])
out = widgets.interactive_output(run_simulation, {
    'grid_size': grid_size_slider,
    'initial_sheep': initial_sheep_slider,
    'initial_wolves': initial_wolves_slider,
    'grass_regrowth_rate': grass_regrowth_slider,
    'sheep_energy_gain': sheep_energy_gain_slider,
    'wolf_energy_gain': wolf_energy_gain_slider,
    'sheep_reproduction_rate': sheep_reproduction_slider,
    'wolf_reproduction_rate': wolf_reproduction_slider,
    'energy_loss_rate': energy_loss_slider,
    'filename': filename_textbox
})

display(ui, out)


Output()

## Iteration 2

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# Parameters for the simulation (with default values)
GRID_SIZE = 50
INITIAL_SHEEP = 150
INITIAL_WOLVES = 20
GRASS_REGROWTH_RATE = 0.06
SHEEP_ENERGY_GAIN = 3
WOLF_ENERGY_GAIN = 8
SHEEP_REPRODUCTION_RATE = 0.06
WOLF_REPRODUCTION_RATE = 0.03
ENERGY_LOSS_RATE = 0.5
SIMULATION_STEPS = 200

# Agent Classes
class Sheep:
    def __init__(self, energy_gain):
        self.x = random.randint(0, GRID_SIZE - 1)
        self.y = random.randint(0, GRID_SIZE - 1)
        self.energy = random.randint(5, 10)
        self.energy_gain = energy_gain

    def move(self):
        self.x = (self.x + random.choice([-1, 0, 1])) % GRID_SIZE
        self.y = (self.y + random.choice([-1, 0, 1])) % GRID_SIZE

    def eat_grass(self, grass):
        if grass[self.x][self.y]:
            grass[self.x][self.y] = False
            self.energy += self.energy_gain

class Wolf:
    def __init__(self, energy_gain):
        self.x = random.randint(0, GRID_SIZE - 1)
        self.y = random.randint(0, GRID_SIZE - 1)
        self.energy = random.randint(10, 20)
        self.energy_gain = energy_gain

    def move(self):
        self.x = (self.x + random.choice([-1, 0, 1])) % GRID_SIZE
        self.y = (self.y + random.choice([-1, 0, 1])) % GRID_SIZE

    def eat_sheep(self, sheep_list):
        for sheep in sheep_list:
            if self.x == sheep.x and self.y == sheep.y:
                sheep_list.remove(sheep)
                self.energy += self.energy_gain
                return True
        return False

# Initialize environment and agents
def initialize(initial_sheep, initial_wolves, sheep_energy_gain, wolf_energy_gain):
    grass = np.random.choice([True, False], (GRID_SIZE, GRID_SIZE), p=[0.5, 0.5])
    sheep_list = [Sheep(sheep_energy_gain) for _ in range(initial_sheep)]
    wolf_list = [Wolf(wolf_energy_gain) for _ in range(initial_wolves)]
    return grass, sheep_list, wolf_list

# Update function
def update(grass, sheep_list, wolf_list, grass_regrowth_rate, sheep_reproduction_rate, wolf_reproduction_rate, energy_loss_rate):
    # Grass regrowth
    grass_regrowth = np.random.choice([True, False], (GRID_SIZE, GRID_SIZE), p=[grass_regrowth_rate, 1 - grass_regrowth_rate])
    grass = np.logical_or(grass, grass_regrowth)

    # Sheep move, eat grass, lose energy, and reproduce
    for sheep in sheep_list[:]:
        sheep.move()
        sheep.eat_grass(grass)
        sheep.energy -= energy_loss_rate
        if sheep.energy <= 0:
            sheep_list.remove(sheep)
        elif random.random() < sheep_reproduction_rate:
            sheep_list.append(Sheep(sheep.energy_gain))

    # Wolves move, eat sheep, lose energy, and reproduce
    for wolf in wolf_list[:]:
        wolf.move()
        if not wolf.eat_sheep(sheep_list):
            wolf.energy -= energy_loss_rate
        if wolf.energy <= 0:
            wolf_list.remove(wolf)
        elif random.random() < wolf_reproduction_rate:
            wolf_list.append(Wolf(wolf.energy_gain))

    return grass, sheep_list, wolf_list

# Simulation
def simulate(steps, grid_size, initial_sheep, initial_wolves, grass_regrowth_rate, sheep_energy_gain, wolf_energy_gain, 
             sheep_reproduction_rate, wolf_reproduction_rate, energy_loss_rate):
    
    global GRID_SIZE
    GRID_SIZE = grid_size

    grass, sheep_list, wolf_list = initialize(initial_sheep, initial_wolves, sheep_energy_gain, wolf_energy_gain)

    sheep_population = []
    wolf_population = []
    grass_coverage = []
    
    for step in range(steps):
        grass, sheep_list, wolf_list = update(grass, sheep_list, wolf_list, grass_regrowth_rate, sheep_reproduction_rate, 
                                              wolf_reproduction_rate, energy_loss_rate)
        sheep_population.append(len(sheep_list))
        wolf_population.append(len(wolf_list))
        grass_coverage.append(np.sum(grass))

        # Log every 10 steps instead of every step
        if step % 10 == 0:
            print(f'Step {step}: Sheep = {len(sheep_list)}, Wolves = {len(wolf_list)}, Grass Coverage = {np.sum(grass)}')

    return sheep_population, wolf_population, grass_coverage

# Plot Population
def plot_population_dynamics(sheep_population, wolf_population, grass_coverage, filename=None):
    plt.figure(figsize=(10, 6))
    plt.plot(sheep_population, label="Sheep Population", color='green')
    plt.plot(wolf_population, label="Wolf Population", color='red')
    plt.plot(grass_coverage, label="Grass Coverage", color='blue')
    plt.xlabel("Time Steps")
    plt.ylabel("Population/Grass Coverage")
    plt.title("Wolf-Sheep Population Dynamics with Grass")
    plt.legend()
    plt.grid()
    
    # Save the figure
    if filename:
        plt.savefig(filename, dpi=300)
        print(f"Plot saved as {filename}")

    plt.show()

# Update the run_simulation func
def run_simulation(grid_size, initial_sheep, initial_wolves, grass_regrowth_rate, sheep_energy_gain, wolf_energy_gain, 
                   sheep_reproduction_rate, wolf_reproduction_rate, energy_loss_rate, filename=None):
    sheep_population, wolf_population, grass_coverage = simulate(SIMULATION_STEPS, grid_size, initial_sheep, initial_wolves, 
                                                                 grass_regrowth_rate, sheep_energy_gain, wolf_energy_gain, 
                                                                 sheep_reproduction_rate, wolf_reproduction_rate, energy_loss_rate)
    plot_population_dynamics(sheep_population, wolf_population, grass_coverage, filename)

# Define sliders
grid_size_slider = widgets.IntSlider(value=50, min=10, max=100, step=1, description='Grid Size')
initial_sheep_slider = widgets.IntSlider(value=150, min=10, max=200, step=1, description='Initial Sheep')
initial_wolves_slider = widgets.IntSlider(value=20, min=5, max=100, step=1, description='Initial Wolves')
grass_regrowth_slider = widgets.FloatSlider(value=0.06, min=0.01, max=0.1, step=0.01, description='Grass Regrowth')
sheep_energy_gain_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description='Sheep Energy Gain')
wolf_energy_gain_slider = widgets.IntSlider(value=8, min=5, max=20, step=1, description='Wolf Energy Gain')
sheep_reproduction_slider = widgets.FloatSlider(value=0.06, min=0.01, max=0.1, step=0.01, description='Sheep Reproduction')
wolf_reproduction_slider = widgets.FloatSlider(value=0.03, min=0.01, max=0.1, step=0.01, description='Wolf Reproduction')
energy_loss_slider = widgets.FloatSlider(value=0.5, min=0.5, max=10, step=0.05, description='Energy Loss')  # Updated here

# Create a filename text input
filename_textbox = widgets.Text(value='population_dynamics_iter2.png', description='Filename:')

ui = widgets.VBox([
    grid_size_slider, initial_sheep_slider, initial_wolves_slider, grass_regrowth_slider, 
    sheep_energy_gain_slider, wolf_energy_gain_slider, sheep_reproduction_slider, 
    wolf_reproduction_slider, energy_loss_slider
])
out = widgets.interactive_output(run_simulation, {
    'grid_size': grid_size_slider,
    'initial_sheep': initial_sheep_slider,
    'initial_wolves': initial_wolves_slider,
    'grass_regrowth_rate': grass_regrowth_slider,
    'sheep_energy_gain': sheep_energy_gain_slider,
    'wolf_energy_gain': wolf_energy_gain_slider,
    'sheep_reproduction_rate': sheep_reproduction_slider,
    'wolf_reproduction_rate': wolf_reproduction_slider,
    'energy_loss_rate': energy_loss_slider,
    'filename': filename_textbox
})

display(ui, out)


Output()